In [1]:
import pandas as pd
import sqlite3

## DB 연동하기

In [2]:
connection = sqlite3.connect('e-commerce.sqlite')
connection

In [ ]:
query = 'select * from sqlite_master'
pd.read_sql(query, connection)

# 기본구문

순서가 중요

* SELECT
* FROM
* WHERE
* GROUP BY
* HAVING
* ORDER BY
* (LIMIT)

## SELECT / FROM

In [ ]:
query = 'SELECT * FROM orders'

pd.read_sql(query, connection)

In [ ]:
query = 'SELECT id, user_id, price FROM orders'

pd.read_sql(query, connection)

## where

In [ ]:
query = """
    SELECT *
    FROM orders
    WHERE state = 'confirmed'
"""

pd.read_sql(query, connection)

In [ ]:
#같지 않다 !=, <>
query = """
    SELECT *
    FROM orders
    WHERE state <> 'confirmed'
"""

pd.read_sql(query, connection)

In [ ]:
query = """
    SELECT *
    FROM orders
    WHERE price <= 500
"""

pd.read_sql(query, connection)

In [ ]:
# IN - 들어있는 지
# NOT IN - 들어있지 않은 지

query = """
    SELECT *
    FROM orders
    WHERE address NOT IN ('Seoul', 'Daejeon')
"""

pd.read_sql(query, connection)

In [ ]:
# between ~ and ~
query = """
    SELECT *
    FROM orders
    WHERE date BETWEEN '2017-01-03' AND '2017-01-09'
"""
pd.read_sql(query,connection)

In [ ]:
# LIKE - 비슷한 형태를 가져오고 싶을 때
# NOT LIKE - LIKE 의 부정
# % 글자 수에 상관 없이 / _ - 자리 수 지정  (___ 3자리)

query = """
    SELECT *
    FROM orders
    WHERE date NOT LIKE '2017-01%'
"""

pd.read_sql(query,connection)

In [ ]:
# state = 'confirmed' & price >=500

query = """
    SELECT *
    FROM orders
    WHERE state = 'confirmed'
        AND price>=500
"""

pd.read_sql(query,connection)

## GROUP BY / HAVING

In [ ]:
#user 별 price 합

query = """
    SELECT 
        user_id, 
        sum(price),
        avg(price),
        min(price),
        max(price)
    FROM orders
    GROUP BY user_id
"""

pd.read_sql(query,connection)

In [ ]:
#user_id, state

query = """
    SELECT
        user_id,
        state,
        sum(price),
        avg(price)
    FROM orders
    GROUP BY user_id, state
"""

pd.read_sql(query,connection)

In [ ]:
#price 합이 1000 이상인 그룹만 남김
# HAVING - 그룹에 대한 조건(GRUOP BY 와 함께 사용된다.)

query = """
    SELECT
        user_id,
        state,
        sum(price) as tot_price,
        avg(price) as avg_price
    FROM orders
    GROUP BY user_id, state
    HAVING tot_price >= 1000
"""

pd.read_sql(query,connection)

## ORDER BY / LIMIT

In [ ]:
query = """
    SELECT *
    FROM orders
    ORDER BY price DESC 
"""

pd.read_sql(query,connection)

In [ ]:
query = """
    SELECT
        user_id,
        state,
        sum(price) as tot_price,
        avg(price) as avg_price
    FROM orders
    GROUP BY user_id, state
    ORDER BY tot_price DESC, user_id
    
"""

pd.read_sql(query,connection)

In [ ]:
#데이터 개수 제한

query = """
    SELECT *
    FROM orders
    LIMIT 5
"""

pd.read_sql(query,connection)

In [ ]:
# 정리 문제
# 'confirmed' data에 대해서, user 별로 price를 집계
# total price 가 500 이상인 user만, price 대해 내림차순으로 정리

query = """
    SELECT 
        user_id,
        sum(price) as tot_price
    FROM orders
    WHERE state = 'confirmed'
    GROUP BY user_id
    HAVING tot_price >=500
    ORDER BY tot_price DESC
"""

pd.read_sql(query,connection)

# 함수

### strftime - 문자에서 시간 정보 추출

* %Y - year
* %m - month
* %d - day
* %H - hour
* %M - minute
* %S - second

In [ ]:
query = """
    SELECT
        date,
        strftime('%Y',date) as year,
        strftime('%Y%m' , date) as yyyymm
    FROM orders
"""

pd.read_sql(query,connection)

## DATE

날짜 연산

In [ ]:
query = """
    SELECT 
        date,
        DATE(date, '+15 days') as confirm_date,
        DATE(date, '+1 month') as confirm_month
    FROM orders
"""

pd.read_sql(query, connection)

### SUBSTR / REPLACE / CAST / CONCAT (||)

In [ ]:
#substr - 문자 일부를 잘라오는 기능 (컬럼명, 시작 인덱스, 끝 인덱스 1부터 시작)
#REPLACE - 데이터에 들어있는 값을 바꿔주는 기능(컬럼명, 원래값, 바꿔줄 값)
#cast - 타입 변환 (컬럼명 as 타입명)
#|| 문자열 붙이기

query = """
    SELECT
        date,
        SUBSTR(date, 1, 4) as year,
        REPLACE(date, '-','') as date2,
        CAST(SUBSTR(date, 1, 4) as int) as year_int,
        SUBSTR(date, 1, 4) || '년' as 년도
    FROM orders
"""

pd.read_sql(query, connection)

# case when
## :~if 문

In [5]:
#price 500 이상이었던 데이터는 "중요"/ "중요 x''

query = """
    SELECT 
        price,
        CASE when price > 500 then '중요'
        else '중요 x'
        END as '중요여부'
    FROM orders
"""

pd.read_sql(query, connection)

,price,중요여부
0,500,중요 x
1,700,중요
2,900,중요
3,500,중요 x
4,700,중요
5,600,중요
6,200,중요 x
7,600,중요
8,200,중요 x
9,650,중요


In [7]:
#서울, 인천 - 수도권 / 대구, 부산 - 경상도 / 나머지 - 기타

query = """
    SELECT 
        address,
        CASE WHEN address IN ('Seoul', 'Incheon') THEN '수도권'
             WHEN address IN ('Daegu' ,'Busan') THEN '경상도'
             ELSE '기타'
        END as 지역
    FROM orders
"""

pd.read_sql(query, connection)

,address,지역
0,Seoul,수도권
1,Seoul,수도권
2,Daejeon,기타
3,nan,기타
4,Incheon,수도권
5,Busan,경상도
6,nan,기타
7,Seoul,수도권
8,Daejeon,기타
9,Daegu,경상도


# join

: inner (둘다 교집합)/ outer(합집합) / left(왼쪽 테이블 기준) / right(오른쪽 테이블 기준) <br>
: 두 개 이상의 테이블을 연결해서 사용하고 싶을 때

In [10]:
query = " SELECT * FROM orders order by user_id"
pd.read_sql(query, connection)

,id,user_id,product_id,date,price,address,state
0,2,1,7,2017-01-03,700,Seoul,confirmed
1,8,1,3,2017-02-04,600,Seoul,confirmed
2,7,2,5,2017-01-10,200,nan,canceled
3,11,2,6,2017-02-28,420,Busan,confirmed
4,1,3,9,2017-01-01,500,Seoul,confirmed
5,3,3,8,2017-01-03,900,Daejeon,confirmed
6,4,4,2,2017-01-07,500,nan,canceled
7,9,4,8,2017-02-12,200,Daejeon,canceled
8,6,5,7,2017-01-09,600,Busan,canceled
9,5,7,3,2017-01-09,700,Incheon,confirmed


In [12]:
query = """
    SELECT *
    FROM users
    """
pd.read_sql(query, connection)

,id,name,gender,age,email
0,1,Kang,male,30,kang@gmail.com
1,2,Kim,female,22,kim@naver.com
2,3,Park,male,37,park@dsschool.co.kr
3,4,Lee,female,15,lee@empas.co.kr
4,5,Son,male,29,son@google.co.uk
5,6,Moon,female,40,moon@hanmail.com
6,7,Choi,male,34,choi@yahoo.com


In [18]:
# inner join - 교집합 from 기준 테이블 (왼쪽) on 기준
query = """
    SELECT orders.*,users.*
    FROM orders 
    INNER JOIN users
        ON orders.user_id = users.id
    ORDER BY user_id
"""

pd.read_sql(query, connection)

,id,user_id,product_id,date,price,address,state,id,name,gender,age,email
0,2,1,7,2017-01-03,700,Seoul,confirmed,1,Kang,male,30,kang@gmail.com
1,8,1,3,2017-02-04,600,Seoul,confirmed,1,Kang,male,30,kang@gmail.com
2,7,2,5,2017-01-10,200,nan,canceled,2,Kim,female,22,kim@naver.com
3,11,2,6,2017-02-28,420,Busan,confirmed,2,Kim,female,22,kim@naver.com
4,1,3,9,2017-01-01,500,Seoul,confirmed,3,Park,male,37,park@dsschool.co.kr
5,3,3,8,2017-01-03,900,Daejeon,confirmed,3,Park,male,37,park@dsschool.co.kr
6,4,4,2,2017-01-07,500,nan,canceled,4,Lee,female,15,lee@empas.co.kr
7,9,4,8,2017-02-12,200,Daejeon,canceled,4,Lee,female,15,lee@empas.co.kr
8,6,5,7,2017-01-09,600,Busan,canceled,5,Son,male,29,son@google.co.uk
9,5,7,3,2017-01-09,700,Incheon,confirmed,7,Choi,male,34,choi@yahoo.com


In [21]:
query = """
    SELECT A.*,
        B.age,
        B.gender
    FROM orders as A
    INNER JOIN users as B
        ON A.user_id = B.id
    ORDER BY user_id
"""

pd.read_sql(query, connection)

,id,user_id,product_id,date,price,address,state,age,gender
0,2,1,7,2017-01-03,700,Seoul,confirmed,30,male
1,8,1,3,2017-02-04,600,Seoul,confirmed,30,male
2,7,2,5,2017-01-10,200,nan,canceled,22,female
3,11,2,6,2017-02-28,420,Busan,confirmed,22,female
4,1,3,9,2017-01-01,500,Seoul,confirmed,37,male
5,3,3,8,2017-01-03,900,Daejeon,confirmed,37,male
6,4,4,2,2017-01-07,500,nan,canceled,15,female
7,9,4,8,2017-02-12,200,Daejeon,canceled,15,female
8,6,5,7,2017-01-09,600,Busan,canceled,29,male
9,5,7,3,2017-01-09,700,Incheon,confirmed,34,male


In [22]:
# INNER JOIN - 교집합

query = """
    SELECT A.*,
        B.age,
        B.gender,
        C.title
    FROM orders as A
    INNER JOIN users as B
        ON A.user_id = B.id
    INNER JOIN products as C
        ON A.product_id = C.id
    ORDER BY user_id
"""

pd.read_sql(query, connection)

,id,user_id,product_id,date,price,address,state,age,gender,title
0,2,1,7,2017-01-03,700,Seoul,confirmed,30,male,pineapple
1,8,1,3,2017-02-04,600,Seoul,confirmed,30,male,banana
2,7,2,5,2017-01-10,200,nan,canceled,22,female,kiwi
3,11,2,6,2017-02-28,420,Busan,confirmed,22,female,melon
4,4,4,2,2017-01-07,500,nan,canceled,15,female,blueberry
5,6,5,7,2017-01-09,600,Busan,canceled,29,male,pineapple
6,5,7,3,2017-01-09,700,Incheon,confirmed,34,male,banana


In [24]:
# INNER JOIN - 교집합

query = """
    SELECT A.*,
        B.age,
        B.gender
    FROM orders as A
    LEFT JOIN users as B
        ON A.user_id = B.id
    ORDER BY user_id
"""

pd.read_sql(query, connection)

,id,user_id,product_id,date,price,address,state,age,gender
0,2,1,7,2017-01-03,700,Seoul,confirmed,30.0,male
1,8,1,3,2017-02-04,600,Seoul,confirmed,30.0,male
2,7,2,5,2017-01-10,200,nan,canceled,22.0,female
3,11,2,6,2017-02-28,420,Busan,confirmed,22.0,female
4,1,3,9,2017-01-01,500,Seoul,confirmed,37.0,male
5,3,3,8,2017-01-03,900,Daejeon,confirmed,37.0,male
6,4,4,2,2017-01-07,500,nan,canceled,15.0,female
7,9,4,8,2017-02-12,200,Daejeon,canceled,15.0,female
8,6,5,7,2017-01-09,600,Busan,canceled,29.0,male
9,5,7,3,2017-01-09,700,Incheon,confirmed,34.0,male


# Subquery

In [28]:
query = """
    SELECT A.*,
        B.gender,
        B.age
    FROM users as A
    INNER JOIN (
        SELECT *
        FROM users
        WHERE gender = 'female'
    
    ) B
        ON A.id = B.id
"""

pd.read_sql(query, connection)

,id,name,gender,age,email,gender,age
0,2,Kim,female,22,kim@naver.com,female,22
1,4,Lee,female,15,lee@empas.co.kr,female,15
2,6,Moon,female,40,moon@hanmail.com,female,40


In [29]:
query = """ 
     WITH F as  (
         SELECT *
         FROM users
         WHERE gender = 'female'
    )
    
    SELECT 
        A.*,
        B.gender,
        B.age
    FROM orders as A
    INNER JOIN F as B
        ON A.user_id = B.id
"""

pd.read_sql(query, connection)

,id,user_id,product_id,date,price,address,state,gender,age
0,4,4,2,2017-01-07,500,nan,canceled,female,15
1,7,2,5,2017-01-10,200,nan,canceled,female,22
2,9,4,8,2017-02-12,200,Daejeon,canceled,female,15
3,11,2,6,2017-02-28,420,Busan,confirmed,female,22


In [31]:
query = """
    SELECT 
        A.*,
        B.gender,
        B.age
    FROM orders as A
    INNER JOIN users as B
        ON A.user_id = B.id
        AND B.gender = 'female'
"""

pd.read_sql(query, connection)

,id,user_id,product_id,date,price,address,state,gender,age
0,4,4,2,2017-01-07,500,nan,canceled,female,15
1,7,2,5,2017-01-10,200,nan,canceled,female,22
2,9,4,8,2017-02-12,200,Daejeon,canceled,female,15
3,11,2,6,2017-02-28,420,Busan,confirmed,female,22


# Window

- window 함수는 원래 데이터의 행 개수가 보존

In [33]:
query = """
    SELECT
        user_id,
        SUM(price)
    FROM orders
    GROUP BY user_id
"""

pd.read_sql(query, connection)

,user_id,SUM(price)
0,1,1300
1,2,620
2,3,1400
3,4,700
4,5,600
5,7,700
6,8,650


In [34]:
query = """
    SELECT 
        user_id,
        SUM(price) OVER(PARTITION BY user_id) as price_sum
    FROM orders
"""

pd.read_sql(query, connection)

,user_id,price_sum
0,1,1300
1,1,1300
2,2,620
3,2,620
4,3,1400
5,3,1400
6,4,700
7,4,700
8,5,600
9,7,700


In [37]:
# rank / row_number
query ="""
    SELECT
        id,
        user_id,
        date,
        RANK() OVER(PaRTITION BY user_id ORDER BY date) as rank
    FROM orders
"""

pd.read_sql(query, connection)

,id,user_id,date,rank
0,2,1,2017-01-03,1
1,8,1,2017-02-04,2
2,7,2,2017-01-10,1
3,11,2,2017-02-28,2
4,1,3,2017-01-01,1
5,3,3,2017-01-03,2
6,4,4,2017-01-07,1
7,9,4,2017-02-12,2
8,6,5,2017-01-09,1
9,5,7,2017-01-09,1
